In [2]:
import numpy as np
import pandas as pd
import sklearn as sk
import pandas as pd
import seaborn as sns
import keras
from keras import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score



d = pd.read_csv('houses_edited.csv')


features = ["bedrooms", "bathrooms", "sqft", "parking", 
            "mean_district_income", "Plex", "Semi-Detached", 
            "Store W/Apt/Offc", "Co-Op Apt", "Co-Ownership Apt", 
            "Link", "Comm Element Condo", "Att/Row/Twnhouse", 
            "Condo Townhouse", "Detached", "Condo Apt"] #define list of features to input in nerual network


df = pd.DataFrame(data = d)


def remove_beds(x): #function to remove the word "beds" from column of number of beds
    x = x.replace(" beds", "")
    return x
def calc_beds(j): #function to calculate number of beds from string
    answer = sum(int(i) for i in j.split('+'))
    return answer


df.bedrooms = df.bedrooms.apply(remove_beds) 
df.bedrooms = df.bedrooms.apply(calc_beds)

In [3]:
df = df.drop(["description", "mls", "bedrooms_ag", "bedrooms_bg", "full_link", "full_address", "title",
              "final_price_transformed", "final_price_log", "city_district", "district_code",
              "index", "lat", "long", "list_price"], axis = 1) #Remove insignificant values

#Fill missing sqft values based on type of property

sqft_Store = 0
sqft_Link = 0
sqft_Twn = 0
sqft_Plex = 0
sqft_Detached = 0
sqft_Semi_detached = 0
sqft_Store_count = 0
sqft_Link_count = 0
sqft_Twn_count = 0
sqft_Plex_count = 0
sqft_Detached_count = 0
sqft_Semi_detached_count = 0


for index, row in df.iterrows():
    if pd.notnull(row["sqft"]) and row["type"] == "Store W/Apt/Offc":
      sqft_Store += row["sqft"]
      sqft_Store_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Link":
      sqft_Link += row["sqft"]
      sqft_Link_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Att/Row/Twnhouse":
      sqft_Twn += row["sqft"]
      sqft_Twn_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Plex":
      sqft_Plex += row["sqft"]
      sqft_Plex_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Detached":
      sqft_Detached += row["sqft"]
      sqft_Detached_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Semi-Detached":
      sqft_Semi_detached += row["sqft"]
      sqft_Semi_detached_count += 1

avg_Store = sqft_Store/sqft_Store_count
avg_Link = sqft_Link/sqft_Link_count
avg_Twn = sqft_Twn/sqft_Twn_count
avg_Plex = sqft_Plex/sqft_Plex_count
avg_Detached = sqft_Detached/sqft_Detached_count
avg_Semi_detached = sqft_Semi_detached/sqft_Semi_detached_count


for index, row in df.iterrows():
  #print(row["type"], row["sqft"])
  if pd.isnull(row["sqft"]) and row["type"] == "Store W/Apt/Offc":
    df.at[index, "sqft"] = avg_Store

  if pd.isnull(row["sqft"]) and row["type"] == "Link":
    df.at[index, "sqft"] = avg_Link

  if pd.isnull(row["sqft"]) and row["type"] == "Att/Row/Twnhouse":
    df.at[index, "sqft"] = avg_Twn

  if pd.isnull(row["sqft"]) and row["type"] == "Plex":
    df.at[index, "sqft"] = avg_Plex

  if pd.isnull(row["sqft"]) and row["type"] == "Detached":
    df.at[index, "sqft"] = avg_Detached

  if pd.isnull(row["sqft"]) and row["type"] == "Semi-Detached":
    df.at[index, "sqft"] = avg_Semi_detached


dummy_type = pd.get_dummies(df["type"]) #allow 
df = pd.concat([df, dummy_type], axis = 1)
df = df.drop(["type"], axis = 1)

In [19]:
train, test = train_test_split(df, test_size = 0.20, random_state = 0) #Split data into 2 parts, 
                                                                       #training and testing data, with ratio 8:2 

x_train = train[features]
y_train = train[['final_price']]

model = Sequential()

model.add(Dense(64, input_dim = 16,activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))

opt = keras.optimizers.Adam(learning_rate = 0.03) 

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['MeanAbsolutePercentageError'])
model.fit(x_train, y_train, epochs = 1500, batch_size = 200, shuffle = False, verbose = 2)
#training = model.predict(x_train)
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy))

predictions = model.predict(x_train)
#print(predictions)
for i in range(10):
  print("%d, expected: %d" % (predictions[i], y_train.iloc[i]))

# Predicting prices from test set
# x_test = test[features]
# y_test = test[['final_price']]


# print(model.evaluate(x_test, y_test))


#Best hyperparameter result
# MSPE: 17.95%
# Neural layers: 64, 32, 1
# Learning rate: 0.03
# Epochs: 1500
# Batch size; 200


Epoch 1/1500
61/61 - 0s - loss: 414577197056.0000 - mean_absolute_percentage_error: 46.3143
Epoch 2/1500
61/61 - 0s - loss: 248939511808.0000 - mean_absolute_percentage_error: 32.5935
Epoch 3/1500
61/61 - 0s - loss: 179445252096.0000 - mean_absolute_percentage_error: 28.1264
Epoch 4/1500
61/61 - 0s - loss: 176612474880.0000 - mean_absolute_percentage_error: 28.4349
Epoch 5/1500
61/61 - 0s - loss: 176442327040.0000 - mean_absolute_percentage_error: 28.3051
Epoch 6/1500
61/61 - 0s - loss: 175747842048.0000 - mean_absolute_percentage_error: 28.1922
Epoch 7/1500
61/61 - 0s - loss: 175478636544.0000 - mean_absolute_percentage_error: 28.1633
Epoch 8/1500
61/61 - 0s - loss: 175362801664.0000 - mean_absolute_percentage_error: 28.1483
Epoch 9/1500
61/61 - 0s - loss: 175285469184.0000 - mean_absolute_percentage_error: 28.1372
Epoch 10/1500
61/61 - 0s - loss: 175226404864.0000 - mean_absolute_percentage_error: 28.1281
Epoch 11/1500
61/61 - 0s - loss: 175177580544.0000 - mean_absolute_percentage_e

Epoch 90/1500
61/61 - 0s - loss: 172331548672.0000 - mean_absolute_percentage_error: 27.6547
Epoch 91/1500
61/61 - 0s - loss: 172296339456.0000 - mean_absolute_percentage_error: 27.6486
Epoch 92/1500
61/61 - 0s - loss: 172262490112.0000 - mean_absolute_percentage_error: 27.6427
Epoch 93/1500
61/61 - 0s - loss: 172226347008.0000 - mean_absolute_percentage_error: 27.6356
Epoch 94/1500
61/61 - 0s - loss: 172189040640.0000 - mean_absolute_percentage_error: 27.6299
Epoch 95/1500
61/61 - 0s - loss: 172151341056.0000 - mean_absolute_percentage_error: 27.6226
Epoch 96/1500
61/61 - 0s - loss: 172108906496.0000 - mean_absolute_percentage_error: 27.6155
Epoch 97/1500
61/61 - 0s - loss: 172066750464.0000 - mean_absolute_percentage_error: 27.6081
Epoch 98/1500
61/61 - 0s - loss: 172033622016.0000 - mean_absolute_percentage_error: 27.6026
Epoch 99/1500
61/61 - 0s - loss: 171992989696.0000 - mean_absolute_percentage_error: 27.5948
Epoch 100/1500
61/61 - 0s - loss: 171952717824.0000 - mean_absolute_pe

Epoch 178/1500
61/61 - 0s - loss: 147693895680.0000 - mean_absolute_percentage_error: 26.9956
Epoch 179/1500
61/61 - 0s - loss: 147443384320.0000 - mean_absolute_percentage_error: 27.0228
Epoch 180/1500
61/61 - 0s - loss: 146619498496.0000 - mean_absolute_percentage_error: 26.8796
Epoch 181/1500
61/61 - 0s - loss: 146600755200.0000 - mean_absolute_percentage_error: 26.8671
Epoch 182/1500
61/61 - 0s - loss: 146665734144.0000 - mean_absolute_percentage_error: 26.8946
Epoch 183/1500
61/61 - 0s - loss: 147011190784.0000 - mean_absolute_percentage_error: 26.8722
Epoch 184/1500
61/61 - 0s - loss: 145751719936.0000 - mean_absolute_percentage_error: 26.7672
Epoch 185/1500
61/61 - 0s - loss: 146082119680.0000 - mean_absolute_percentage_error: 26.8154
Epoch 186/1500
61/61 - 0s - loss: 145883463680.0000 - mean_absolute_percentage_error: 26.7826
Epoch 187/1500
61/61 - 0s - loss: 145221828608.0000 - mean_absolute_percentage_error: 26.6292
Epoch 188/1500
61/61 - 0s - loss: 144963616768.0000 - mean_a

61/61 - 0s - loss: 125983154176.0000 - mean_absolute_percentage_error: 22.5610
Epoch 266/1500
61/61 - 0s - loss: 129772331008.0000 - mean_absolute_percentage_error: 23.2022
Epoch 267/1500
61/61 - 0s - loss: 127893544960.0000 - mean_absolute_percentage_error: 22.5367
Epoch 268/1500
61/61 - 0s - loss: 129759674368.0000 - mean_absolute_percentage_error: 22.7660
Epoch 269/1500
61/61 - 0s - loss: 127354585088.0000 - mean_absolute_percentage_error: 22.7168
Epoch 270/1500
61/61 - 0s - loss: 127873900544.0000 - mean_absolute_percentage_error: 22.5991
Epoch 271/1500
61/61 - 0s - loss: 129245765632.0000 - mean_absolute_percentage_error: 22.5841
Epoch 272/1500
61/61 - 0s - loss: 127363194880.0000 - mean_absolute_percentage_error: 22.6840
Epoch 273/1500
61/61 - 0s - loss: 127692062720.0000 - mean_absolute_percentage_error: 22.6446
Epoch 274/1500
61/61 - 0s - loss: 128765779968.0000 - mean_absolute_percentage_error: 22.6112
Epoch 275/1500
61/61 - 0s - loss: 130893201408.0000 - mean_absolute_percent

Epoch 353/1500
61/61 - 0s - loss: 125178413056.0000 - mean_absolute_percentage_error: 22.2368
Epoch 354/1500
61/61 - 0s - loss: 122601357312.0000 - mean_absolute_percentage_error: 22.0451
Epoch 355/1500
61/61 - 0s - loss: 121376931840.0000 - mean_absolute_percentage_error: 21.9388
Epoch 356/1500
61/61 - 0s - loss: 123280089088.0000 - mean_absolute_percentage_error: 22.0545
Epoch 357/1500
61/61 - 0s - loss: 122600849408.0000 - mean_absolute_percentage_error: 22.3330
Epoch 358/1500
61/61 - 1s - loss: 123135057920.0000 - mean_absolute_percentage_error: 22.1375
Epoch 359/1500
61/61 - 0s - loss: 123840856064.0000 - mean_absolute_percentage_error: 22.0149
Epoch 360/1500
61/61 - 0s - loss: 125076045824.0000 - mean_absolute_percentage_error: 22.6011
Epoch 361/1500
61/61 - 0s - loss: 121846284288.0000 - mean_absolute_percentage_error: 21.6819
Epoch 362/1500
61/61 - 0s - loss: 124769665024.0000 - mean_absolute_percentage_error: 22.1294
Epoch 363/1500
61/61 - 0s - loss: 123445329920.0000 - mean_a

...

Epoch 1139/1500
61/61 - 0s - loss: 113243136000.0000 - mean_absolute_percentage_error: 21.1164
Epoch 1140/1500
61/61 - 0s - loss: 110294671360.0000 - mean_absolute_percentage_error: 21.0500
Epoch 1141/1500
61/61 - 0s - loss: 120153669632.0000 - mean_absolute_percentage_error: 22.8161
Epoch 1142/1500
61/61 - 0s - loss: 122377109504.0000 - mean_absolute_percentage_error: 22.9897
Epoch 1143/1500
61/61 - 0s - loss: 119235379200.0000 - mean_absolute_percentage_error: 22.1111
Epoch 1144/1500
61/61 - 0s - loss: 118478348288.0000 - mean_absolute_percentage_error: 22.0542
Epoch 1145/1500
61/61 - 0s - loss: 118106292224.0000 - mean_absolute_percentage_error: 21.7964
Epoch 1146/1500
61/61 - 0s - loss: 122900299776.0000 - mean_absolute_percentage_error: 22.3809
Epoch 1147/1500
61/61 - 0s - loss: 106925547520.0000 - mean_absolute_percentage_error: 20.5906
Epoch 1148/1500
61/61 - 0s - loss: 108932923392.0000 - mean_absolute_percentage_error: 20.7210
Epoch 1149/1500
61/61 - 0s - loss: 116986920960.00

Epoch 1226/1500
61/61 - 0s - loss: 107542691840.0000 - mean_absolute_percentage_error: 20.8148
Epoch 1227/1500
61/61 - 0s - loss: 119781089280.0000 - mean_absolute_percentage_error: 22.9979
Epoch 1228/1500
61/61 - 0s - loss: 121007210496.0000 - mean_absolute_percentage_error: 22.7530
Epoch 1229/1500
61/61 - 0s - loss: 117926109184.0000 - mean_absolute_percentage_error: 22.3716
Epoch 1230/1500
61/61 - 0s - loss: 114521661440.0000 - mean_absolute_percentage_error: 22.3068
Epoch 1231/1500
61/61 - 0s - loss: 118383140864.0000 - mean_absolute_percentage_error: 22.8564
Epoch 1232/1500
61/61 - 0s - loss: 110197481472.0000 - mean_absolute_percentage_error: 20.9803
Epoch 1233/1500
61/61 - 0s - loss: 106221412352.0000 - mean_absolute_percentage_error: 20.8025
Epoch 1234/1500
61/61 - 0s - loss: 120172281856.0000 - mean_absolute_percentage_error: 22.9349
Epoch 1235/1500
61/61 - 0s - loss: 121562210304.0000 - mean_absolute_percentage_error: 22.5018
Epoch 1236/1500
61/61 - 0s - loss: 110717665280.00

Epoch 1313/1500
61/61 - 0s - loss: 116636016640.0000 - mean_absolute_percentage_error: 22.2925
Epoch 1314/1500
61/61 - 0s - loss: 124231737344.0000 - mean_absolute_percentage_error: 22.3381
Epoch 1315/1500
61/61 - 0s - loss: 107683274752.0000 - mean_absolute_percentage_error: 20.5701
Epoch 1316/1500
61/61 - 0s - loss: 116247486464.0000 - mean_absolute_percentage_error: 22.0459
Epoch 1317/1500
61/61 - 0s - loss: 120818049024.0000 - mean_absolute_percentage_error: 22.8410
Epoch 1318/1500
61/61 - 0s - loss: 121640681472.0000 - mean_absolute_percentage_error: 22.3974
Epoch 1319/1500
61/61 - 0s - loss: 121548398592.0000 - mean_absolute_percentage_error: 23.3492
Epoch 1320/1500
61/61 - 0s - loss: 127314198528.0000 - mean_absolute_percentage_error: 23.3700
Epoch 1321/1500
61/61 - 0s - loss: 113913847808.0000 - mean_absolute_percentage_error: 21.1911
Epoch 1322/1500
61/61 - 0s - loss: 125914390528.0000 - mean_absolute_percentage_error: 22.8111
Epoch 1323/1500
61/61 - 0s - loss: 115263062016.00

Epoch 1400/1500
61/61 - 0s - loss: 130049900544.0000 - mean_absolute_percentage_error: 23.3327
Epoch 1401/1500
61/61 - 0s - loss: 128681443328.0000 - mean_absolute_percentage_error: 22.8833
Epoch 1402/1500
61/61 - 0s - loss: 129241309184.0000 - mean_absolute_percentage_error: 23.3717
Epoch 1403/1500
61/61 - 0s - loss: 119361716224.0000 - mean_absolute_percentage_error: 21.8776
Epoch 1404/1500
61/61 - 0s - loss: 115899973632.0000 - mean_absolute_percentage_error: 21.1466
Epoch 1405/1500
61/61 - 0s - loss: 125068664832.0000 - mean_absolute_percentage_error: 22.7809
Epoch 1406/1500
61/61 - 0s - loss: 118537658368.0000 - mean_absolute_percentage_error: 21.7545
Epoch 1407/1500
61/61 - 0s - loss: 120407498752.0000 - mean_absolute_percentage_error: 21.8601
Epoch 1408/1500
61/61 - 0s - loss: 133513953280.0000 - mean_absolute_percentage_error: 22.8384
Epoch 1409/1500
61/61 - 0s - loss: 119310958592.0000 - mean_absolute_percentage_error: 22.1969
Epoch 1410/1500
61/61 - 0s - loss: 119203037184.00

Epoch 1487/1500
61/61 - 0s - loss: 129761083392.0000 - mean_absolute_percentage_error: 23.6423
Epoch 1488/1500
61/61 - 0s - loss: 123829788672.0000 - mean_absolute_percentage_error: 23.0244
Epoch 1489/1500
61/61 - 0s - loss: 115212918784.0000 - mean_absolute_percentage_error: 21.1010
Epoch 1490/1500
61/61 - 0s - loss: 111275540480.0000 - mean_absolute_percentage_error: 21.0889
Epoch 1491/1500
61/61 - 0s - loss: 125785858048.0000 - mean_absolute_percentage_error: 22.8032
Epoch 1492/1500
61/61 - 0s - loss: 116380082176.0000 - mean_absolute_percentage_error: 21.3011
Epoch 1493/1500
61/61 - 0s - loss: 115806265344.0000 - mean_absolute_percentage_error: 22.0685
Epoch 1494/1500
61/61 - 0s - loss: 115052756992.0000 - mean_absolute_percentage_error: 21.9149
Epoch 1495/1500
61/61 - 0s - loss: 116740677632.0000 - mean_absolute_percentage_error: 21.7354
Epoch 1496/1500
61/61 - 0s - loss: 116739432448.0000 - mean_absolute_percentage_error: 22.2990
Epoch 1497/1500
61/61 - 0s - loss: 116747059200.00